Attempted to push results to github but it resulted in filesize being too big (>100MB). Therefore, will try to save to drive instead, as downloading the file is ineffective. Github Credential Helper is likely redundant.

Idea: zip everything but the weights, as they are the only thing really using storage. The rest can quickly be downloaded and evaluated, independently. Weights also need to be obtained, so drive is still wanted.

In [1]:
import os
HOME = '/content'

In [2]:
from google.colab import drive
DRIVE = HOME+'/drive'
drive.mount(DRIVE)
DRIVE = DRIVE+'/MyDrive'

Mounted at /content/drive


In [3]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
!nvidia-smi

Thu May 30 12:13:11 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install yolov9 and pretrained weights

In [5]:
!git clone https://github.com/Hallvaeb/yolov9-masterthesis.git

Cloning into 'yolov9-masterthesis'...
remote: Enumerating objects: 163, done.
remote: Counting objects: 100% (163/163), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 163 (delta 50), reused 148 (delta 39), pack-reused 0
Receiving objects: 100% (163/163), 581.46 KiB | 15.30 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [6]:
%cd yolov9-masterthesis
!pip install -r requirements.txt -q

/content/yolov9-masterthesis
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.3 MB/s eta 0:00:00


## Download and prepare the dataset for training

In [8]:
!mkdir -p {HOME}/dataset

In [ ]:
!unzip -n -q {DRIVE}/datasets/football-players-yolo.zip -d {HOME}/dataset

In [ ]:
import os
import random

labels_path = '/content/dataset/football-players-yolo/labels'
output_path = '/content/dataset/football-players-yolo'

# Ratios for splitting the datasets
train_ratio = 1
val_ratio = 0

# test_ratio is implicitly determined

# Get all file names without their extensions
filenames = [os.path.splitext(file)[0] for file in os.listdir(labels_path) if os.path.isfile(os.path.join(labels_path, file))]

# Shuffle the list of filenames to ensure random distribution
random.shuffle(filenames)

# Calculate split indices
no_total_files = len(filenames)
train_end = int(no_total_files * train_ratio)
print(train_end)
print(no_total_files)

if(no_total_files == train_end):
  train_end-=1

val_end = train_end + int(no_total_files * val_ratio) +1

# Split the filenames
train_filenames = filenames[:train_end]
val_filenames = filenames[train_end:val_end]
test_filenames = filenames[val_end:]
print(val_filenames)

# Function to write filenames to a file
def write_filenames_to_file(filenames, file_path):
    with open(file_path, 'w') as file:
        for name in filenames:
            file.write(f'./images/{name}.jpg\n')

# Write the splits to their respective files
write_filenames_to_file(train_filenames, os.path.join(output_path, 'train.txt'))
write_filenames_to_file(val_filenames, os.path.join(output_path, 'val.txt'))
write_filenames_to_file(test_filenames, os.path.join(output_path, 'test.txt'))

print("Files have been split and saved successfully.")

663
663
['538438_1_10_png.rf.394f8e55b797bda34d8c5600bc236474']
Files have been split and saved successfully.


Now ensure yolov9-masterthesis/data.yaml is pointing to the right train and val txt files.

## Train the model
Prior to running this, /models/detect/yolov9-e.yaml was modified to have nc: 1. This file is responsible for the detector architecture. We freeze the backbone, to shorten training time.

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch -1 \
--epochs 5 \
--img 640 \
--min-items 0 \
--data /content/dataset/football-players-yolo/data.yaml \
--cfg yolov9-e.yaml \
--project . \
--single-cls \
--noval \
--weights /content/weights/yolov9-e.pt \
--freeze 28

# --device cpu \
# --close-mosaic 15 \

/content
2024-05-29 06:17:54.891639: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 06:17:54.891687: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 06:17:54.893105: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 06:17:54.900365: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 06:17:56.089820: W tensorflow/com

In [ ]:
results_saved_to = "exp"
!zip -r {DRIVE}/football-5e.zip {HOME}/$results_saved_to

  adding: content/exp/ (stored 0%)
  adding: content/exp/train_batch0.jpg (deflated 7%)
  adding: content/exp/F1_curve.png (deflated 16%)
  adding: content/exp/labels.jpg (deflated 29%)
  adding: content/exp/weights/ (stored 0%)
  adding: content/exp/weights/last.pt (deflated 8%)
  adding: content/exp/weights/best.pt (deflated 8%)
  adding: content/exp/R_curve.png (deflated 17%)
  adding: content/exp/events.out.tfevents.1716963477.faccc4e149c6.2768.0 (deflated 19%)
  adding: content/exp/confusion_matrix.png (deflated 41%)
  adding: content/exp/labels_correlogram.jpg (deflated 30%)
  adding: content/exp/P_curve.png (deflated 23%)
  adding: content/exp/PR_curve.png (deflated 29%)
  adding: content/exp/hyp.yaml (deflated 43%)
  adding: content/exp/train_batch1.jpg (deflated 16%)
  adding: content/exp/train_batch2.jpg (deflated 11%)
  adding: content/exp/results.csv (deflated 84%)
  adding: content/exp/val_batch0_labels.jpg (deflated 7%)
  adding: content/exp/val_batch0_pred.jpg (deflated 

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 5 \
--img 640 \
--min-items 0 \
--data /content/dataset/football-players-yolo/data.yaml \
--cfg yolov9-e.yaml \
--project . \
--single-cls \
--noval \
--weights /content/exp/weights/best.pt \
--freeze 28

# --weights /content/weights/yolov9-e.pt \
# --device cpu \
# --close-mosaic 15 \

/content
2024-05-29 06:26:51.313090: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 06:26:51.313141: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 06:26:51.314446: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 06:26:51.321842: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 06:26:52.483608: W tensorflow/com

In [ ]:
results_saved_to = "exp3"
!zip -r {DRIVE}/football-10e.zip {HOME}/$results_saved_to

  adding: content/exp3/ (stored 0%)
  adding: content/exp3/train_batch0.jpg (deflated 7%)
  adding: content/exp3/F1_curve.png (deflated 17%)
  adding: content/exp3/labels.jpg (deflated 29%)
  adding: content/exp3/weights/ (stored 0%)
  adding: content/exp3/weights/last.pt (deflated 8%)
  adding: content/exp3/weights/best.pt (deflated 8%)
  adding: content/exp3/R_curve.png (deflated 17%)
  adding: content/exp3/events.out.tfevents.1716964013.faccc4e149c6.5438.0 (deflated 19%)
  adding: content/exp3/confusion_matrix.png (deflated 41%)
  adding: content/exp3/labels_correlogram.jpg (deflated 30%)
  adding: content/exp3/P_curve.png (deflated 24%)
  adding: content/exp3/PR_curve.png (deflated 30%)
  adding: content/exp3/hyp.yaml (deflated 43%)
  adding: content/exp3/train_batch1.jpg (deflated 16%)
  adding: content/exp3/train_batch2.jpg (deflated 11%)
  adding: content/exp3/results.csv (deflated 83%)
  adding: content/exp3/val_batch0_labels.jpg (deflated 7%)
  adding: content/exp3/val_batch0_

## PRW

In [ ]:
!unzip -n -q {DRIVE}/datasets/PRW-yolo.zip -d {HOME}/dataset

In [ ]:
import os
import random

labels_path = '/content/dataset/PRW-yolo/labels'
output_path = '/content/dataset/PRW-yolo'

# Ratios for splitting the datasets
train_ratio = 1
val_ratio = 0

# test_ratio is implicitly determined

# Get all file names without their extensions
filenames = [os.path.splitext(file)[0] for file in os.listdir(labels_path) if os.path.isfile(os.path.join(labels_path, file))]

# Shuffle the list of filenames to ensure random distribution
random.shuffle(filenames)

# Calculate split indices
no_total_files = len(filenames)
train_end = int(no_total_files * train_ratio)
print(train_end)
print(no_total_files)

if(no_total_files == train_end):
  train_end-=1

val_end = train_end + int(no_total_files * val_ratio) +1

# Split the filenames
train_filenames = filenames[:train_end]
val_filenames = filenames[train_end:val_end]
test_filenames = filenames[val_end:]
print(val_filenames)

# Function to write filenames to a file
def write_filenames_to_file(filenames, file_path):
    with open(file_path, 'w') as file:
        for name in filenames:
            file.write(f'./images/{name}.jpg\n')

# Write the splits to their respective files
write_filenames_to_file(train_filenames, os.path.join(output_path, 'train.txt'))
write_filenames_to_file(val_filenames, os.path.join(output_path, 'val.txt'))
write_filenames_to_file(test_filenames, os.path.join(output_path, 'test.txt'))

print("Files have been split and saved successfully.")

11816
11816
['c3s1_117783']
Files have been split and saved successfully.


In [ ]:
!cp /content/drive/MyDrive/datasets/PRW-5e.pt /content/weights/PRW-5e.pt

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch -1 \
--epochs 5 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data.yaml \
--cfg yolov9-e.yaml \
--project . \
--single-cls \
--noval \
--weights /content/weights/PRW-5e.pt \
--freeze 28

# --device cpu \
# --close-mosaic 15 \

/content
2024-05-29 06:39:30.465371: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 06:39:30.465427: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 06:39:30.466927: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-29 06:39:30.474190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-29 06:39:31.639946: W tensorflow/com

In [ ]:
results_saved_to = "exp4"
!zip -r {DRIVE}/PRW-5e.zip {HOME}/$results_saved_to

  adding: content/exp4/ (stored 0%)
  adding: content/exp4/train_batch0.jpg (deflated 7%)
  adding: content/exp4/events.out.tfevents.1716964772.faccc4e149c6.8833.0 (deflated 18%)
  adding: content/exp4/F1_curve.png (deflated 17%)
  adding: content/exp4/labels.jpg (deflated 35%)
  adding: content/exp4/weights/ (stored 0%)
  adding: content/exp4/weights/last.pt (deflated 8%)
  adding: content/exp4/weights/best.pt (deflated 8%)
  adding: content/exp4/R_curve.png (deflated 19%)
  adding: content/exp4/confusion_matrix.png (deflated 43%)
  adding: content/exp4/labels_correlogram.jpg (deflated 33%)
  adding: content/exp4/P_curve.png (deflated 21%)
  adding: content/exp4/PR_curve.png (deflated 29%)
  adding: content/exp4/hyp.yaml (deflated 43%)
  adding: content/exp4/train_batch1.jpg (deflated 17%)
  adding: content/exp4/train_batch2.jpg (deflated 11%)
  adding: content/exp4/results.csv (deflated 83%)
  adding: content/exp4/val_batch0_labels.jpg (deflated 7%)
  adding: content/exp4/val_batch0_

# FIMUS resume training until 50 and 100 epochs

In [9]:
!unzip -n -q {DRIVE}/FIMUSDataset/Inconsistent.zip -d {HOME}/dataset

In [10]:
import os
import random

labels_path = '/content/dataset/Inconsistent/labels'
output_path = '/content/dataset/Inconsistent'

# Ratios for splitting the datasets
train_ratio = 1
val_ratio = 0

# test_ratio is implicitly determined

# Get all file names without their extensions
filenames = [os.path.splitext(file)[0] for file in os.listdir(labels_path) if os.path.isfile(os.path.join(labels_path, file))]

# Shuffle the list of filenames to ensure random distribution
random.shuffle(filenames)

# Calculate split indices
no_total_files = len(filenames)
train_end = int(no_total_files * train_ratio)
print(train_end)
print(no_total_files)

if(no_total_files == train_end):
  train_end-=1

val_end = train_end + int(no_total_files * val_ratio) +1

# Split the filenames
train_filenames = filenames[:train_end]
val_filenames = filenames[train_end:val_end]
test_filenames = filenames[val_end:]
print(val_filenames)

# Function to write filenames to a file
def write_filenames_to_file(filenames, file_path):
    with open(file_path, 'w') as file:
        for name in filenames:
            file.write(f'./images/{name}.jpg\n')

# Write the splits to their respective files
write_filenames_to_file(train_filenames, os.path.join(output_path, 'train.txt'))
write_filenames_to_file(val_filenames, os.path.join(output_path, 'val.txt'))
write_filenames_to_file(test_filenames, os.path.join(output_path, 'test.txt'))

print("Files have been split and saved successfully.")

2637
2637
['120324-164120-right']
Files have been split and saved successfully.


In [11]:
!cp /content/drive/MyDrive/FIMUSDataset/Inconsistent-50e.pt /content/weights/Inconsistent-50e.pt

In [19]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 10 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data.yaml \
--cfg yolov9-e.yaml \
--project /content/80e \
--single-cls \
--noval \
--weights /content/70e/exp/weights/best.pt \
--freeze 28

# --device cpu \
# --close-mosaic 15 \

/content
2024-05-30 14:26:33.523083: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 14:26:33.523155: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 14:26:33.524729: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 14:26:33.532259: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 14:26:34.709876: W tensor

In [20]:
results_saved_to = "80e/exp"
!zip -r {DRIVE}/80e.zip {HOME}/$results_saved_to

  adding: content/80e/exp/ (stored 0%)
  adding: content/80e/exp/confusion_matrix.png (deflated 43%)
  adding: content/80e/exp/train_batch1.jpg (deflated 25%)
  adding: content/80e/exp/labels_correlogram.jpg (deflated 41%)
  adding: content/80e/exp/R_curve.png (deflated 24%)
  adding: content/80e/exp/PR_curve.png (deflated 29%)
  adding: content/80e/exp/weights/ (stored 0%)
  adding: content/80e/exp/weights/best.pt (deflated 8%)
  adding: content/80e/exp/weights/last.pt (deflated 8%)
  adding: content/80e/exp/val_batch0_pred.jpg (deflated 22%)
  adding: content/80e/exp/hyp.yaml (deflated 43%)
  adding: content/80e/exp/train_batch0.jpg (deflated 14%)
  adding: content/80e/exp/labels.jpg (deflated 33%)
  adding: content/80e/exp/P_curve.png (deflated 24%)
  adding: content/80e/exp/opt.yaml (deflated 50%)
  adding: content/80e/exp/results.csv (deflated 87%)
  adding: content/80e/exp/train_batch2.jpg (deflated 21%)
  adding: content/80e/exp/F1_curve.png (deflated 20%)
  adding: content/80e/

In [21]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 10 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data.yaml \
--cfg yolov9-e.yaml \
--project /content/90e \
--single-cls \
--noval \
--weights /content/80e/exp/weights/best.pt \
--freeze 28

# --device cpu \
# --close-mosaic 15 \

/content
2024-05-30 15:15:29.183726: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 15:15:29.183779: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 15:15:29.185115: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 15:15:29.192802: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 15:15:30.448814: W tensor

In [22]:
results_saved_to = "90e/exp"
!zip -r {DRIVE}/90e.zip {HOME}/$results_saved_to

  adding: content/90e/exp/ (stored 0%)
  adding: content/90e/exp/confusion_matrix.png (deflated 43%)
  adding: content/90e/exp/train_batch1.jpg (deflated 25%)
  adding: content/90e/exp/labels_correlogram.jpg (deflated 41%)
  adding: content/90e/exp/R_curve.png (deflated 22%)
  adding: content/90e/exp/PR_curve.png (deflated 30%)
  adding: content/90e/exp/weights/ (stored 0%)
  adding: content/90e/exp/weights/best.pt (deflated 8%)
  adding: content/90e/exp/weights/last.pt (deflated 8%)
  adding: content/90e/exp/val_batch0_pred.jpg (deflated 22%)
  adding: content/90e/exp/hyp.yaml (deflated 43%)
  adding: content/90e/exp/events.out.tfevents.1717082131.68b1c5c90573.47525.0 (deflated 20%)
  adding: content/90e/exp/train_batch0.jpg (deflated 14%)
  adding: content/90e/exp/labels.jpg (deflated 33%)
  adding: content/90e/exp/P_curve.png (deflated 22%)
  adding: content/90e/exp/opt.yaml (deflated 50%)
  adding: content/90e/exp/results.csv (deflated 87%)
  adding: content/90e/exp/train_batch2.jp

In [23]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 10 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data.yaml \
--cfg yolov9-e.yaml \
--project /content/100e \
--single-cls \
--noval \
--weights /content/90e/exp/weights/best.pt \
--freeze 28

/content
2024-05-30 16:03:13.759843: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 16:03:13.759900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 16:03:13.761405: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 16:03:13.768486: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 16:03:14.940585: W tensor

In [24]:
results_saved_to = "100e/exp"
!zip -r {DRIVE}/100e.zip {HOME}/$results_saved_to

  adding: content/100e/exp/ (stored 0%)
  adding: content/100e/exp/confusion_matrix.png (deflated 43%)
  adding: content/100e/exp/events.out.tfevents.1717084996.68b1c5c90573.59839.0 (deflated 20%)
  adding: content/100e/exp/train_batch1.jpg (deflated 25%)
  adding: content/100e/exp/labels_correlogram.jpg (deflated 41%)
  adding: content/100e/exp/R_curve.png (deflated 23%)
  adding: content/100e/exp/PR_curve.png (deflated 27%)
  adding: content/100e/exp/weights/ (stored 0%)
  adding: content/100e/exp/weights/best.pt (deflated 8%)
  adding: content/100e/exp/weights/last.pt (deflated 8%)
  adding: content/100e/exp/val_batch0_pred.jpg (deflated 22%)
  adding: content/100e/exp/hyp.yaml (deflated 43%)
  adding: content/100e/exp/train_batch0.jpg (deflated 14%)
  adding: content/100e/exp/labels.jpg (deflated 33%)
  adding: content/100e/exp/P_curve.png (deflated 21%)
  adding: content/100e/exp/opt.yaml (deflated 50%)
  adding: content/100e/exp/results.csv (deflated 87%)
  adding: content/100e/e

In [25]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 25 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data.yaml \
--cfg yolov9-e.yaml \
--project /content/125e \
--single-cls \
--noval \
--weights /content/100e/exp/weights/best.pt \
--freeze 28

/content
2024-05-30 16:50:34.260224: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 16:50:34.260274: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 16:50:34.261611: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 16:50:34.270221: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 16:50:35.554702: W tensor

In [26]:
results_saved_to = "125e/exp"
!zip -r {DRIVE}/125e.zip {HOME}/$results_saved_to

  adding: content/125e/exp/ (stored 0%)
  adding: content/125e/exp/confusion_matrix.png (deflated 43%)
  adding: content/125e/exp/train_batch1.jpg (deflated 25%)
  adding: content/125e/exp/labels_correlogram.jpg (deflated 41%)
  adding: content/125e/exp/R_curve.png (deflated 23%)
  adding: content/125e/exp/PR_curve.png (deflated 27%)
  adding: content/125e/exp/weights/ (stored 0%)
  adding: content/125e/exp/weights/best.pt (deflated 8%)
  adding: content/125e/exp/weights/last.pt (deflated 8%)
  adding: content/125e/exp/val_batch0_pred.jpg (deflated 22%)
  adding: content/125e/exp/hyp.yaml (deflated 43%)
  adding: content/125e/exp/events.out.tfevents.1717087836.68b1c5c90573.72056.0 (deflated 21%)
  adding: content/125e/exp/train_batch0.jpg (deflated 14%)
  adding: content/125e/exp/labels.jpg (deflated 33%)
  adding: content/125e/exp/P_curve.png (deflated 22%)
  adding: content/125e/exp/opt.yaml (deflated 50%)
  adding: content/125e/exp/results.csv (deflated 90%)
  adding: content/125e/e

In [27]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 25 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data.yaml \
--cfg yolov9-e.yaml \
--project /content/150e \
--single-cls \
--noval \
--weights /content/125e/exp/weights/best.pt \
--freeze 28

/content
2024-05-30 18:48:08.338077: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 18:48:08.338129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 18:48:08.339569: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 18:48:08.347273: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 18:48:09.527928: W tensor

In [28]:
results_saved_to = "150e/exp"
!zip -r {DRIVE}/150e.zip {HOME}/$results_saved_to

  adding: content/150e/exp/ (stored 0%)
  adding: content/150e/exp/confusion_matrix.png (deflated 43%)
  adding: content/150e/exp/train_batch1.jpg (deflated 25%)
  adding: content/150e/exp/events.out.tfevents.1717094890.68b1c5c90573.102163.0 (deflated 21%)
  adding: content/150e/exp/labels_correlogram.jpg (deflated 41%)
  adding: content/150e/exp/R_curve.png (deflated 23%)
  adding: content/150e/exp/PR_curve.png (deflated 30%)
  adding: content/150e/exp/weights/ (stored 0%)
  adding: content/150e/exp/weights/best.pt (deflated 8%)
  adding: content/150e/exp/weights/last.pt (deflated 8%)
  adding: content/150e/exp/val_batch0_pred.jpg (deflated 22%)
  adding: content/150e/exp/hyp.yaml (deflated 43%)
  adding: content/150e/exp/train_batch0.jpg (deflated 14%)
  adding: content/150e/exp/labels.jpg (deflated 33%)
  adding: content/150e/exp/P_curve.png (deflated 22%)
  adding: content/150e/exp/opt.yaml (deflated 50%)
  adding: content/150e/exp/results.csv (deflated 90%)
  adding: content/150e/

In [29]:
!cp /content/drive/MyDrive/FIMUSDataset/Inconsistent-20e.pt /content/weights/Inconsistent-20e.pt

In [30]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 10 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data.yaml \
--cfg yolov9-e.yaml \
--project /content/30e \
--single-cls \
--noval \
--weights /content/weights/Inconsistent-20e.pt \
--freeze 28

/content
2024-05-30 20:45:44.849381: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 20:45:44.849440: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 20:45:44.851001: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 20:45:44.859048: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 20:45:45.977668: W tensor

In [31]:
results_saved_to = "30e/exp"
!zip -r {DRIVE}/30e.zip {HOME}/$results_saved_to

  adding: content/30e/exp/ (stored 0%)
  adding: content/30e/exp/confusion_matrix.png (deflated 43%)
  adding: content/30e/exp/train_batch1.jpg (deflated 25%)
  adding: content/30e/exp/labels_correlogram.jpg (deflated 41%)
  adding: content/30e/exp/R_curve.png (deflated 23%)
  adding: content/30e/exp/PR_curve.png (deflated 29%)
  adding: content/30e/exp/weights/ (stored 0%)
  adding: content/30e/exp/weights/best.pt (deflated 8%)
  adding: content/30e/exp/weights/last.pt (deflated 8%)
  adding: content/30e/exp/val_batch0_pred.jpg (deflated 22%)
  adding: content/30e/exp/events.out.tfevents.1717101946.68b1c5c90573.132286.0 (deflated 20%)
  adding: content/30e/exp/hyp.yaml (deflated 43%)
  adding: content/30e/exp/train_batch0.jpg (deflated 14%)
  adding: content/30e/exp/labels.jpg (deflated 33%)
  adding: content/30e/exp/P_curve.png (deflated 22%)
  adding: content/30e/exp/opt.yaml (deflated 50%)
  adding: content/30e/exp/results.csv (deflated 87%)
  adding: content/30e/exp/train_batch2.j

In [32]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 10 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data.yaml \
--cfg yolov9-e.yaml \
--project /content/40e \
--single-cls \
--noval \
--weights /content/30e/exp/weights/best.pt \
--freeze 28

/content
2024-05-30 21:35:07.957411: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 21:35:07.957466: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 21:35:07.958832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 21:35:07.966215: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-30 21:35:09.163787: W tensor

In [33]:
results_saved_to = "40e/exp"
!zip -r {DRIVE}/40e.zip {HOME}/$results_saved_to

  adding: content/40e/exp/ (stored 0%)
  adding: content/40e/exp/confusion_matrix.png (deflated 43%)
  adding: content/40e/exp/train_batch1.jpg (deflated 25%)
  adding: content/40e/exp/labels_correlogram.jpg (deflated 41%)
  adding: content/40e/exp/R_curve.png (deflated 24%)
  adding: content/40e/exp/PR_curve.png (deflated 29%)
  adding: content/40e/exp/events.out.tfevents.1717104910.68b1c5c90573.145018.0 (deflated 20%)
  adding: content/40e/exp/weights/ (stored 0%)
  adding: content/40e/exp/weights/best.pt (deflated 8%)
  adding: content/40e/exp/weights/last.pt (deflated 8%)
  adding: content/40e/exp/val_batch0_pred.jpg (deflated 22%)
  adding: content/40e/exp/hyp.yaml (deflated 43%)
  adding: content/40e/exp/train_batch0.jpg (deflated 14%)
  adding: content/40e/exp/labels.jpg (deflated 33%)
  adding: content/40e/exp/P_curve.png (deflated 22%)
  adding: content/40e/exp/opt.yaml (deflated 50%)
  adding: content/40e/exp/results.csv (deflated 87%)
  adding: content/40e/exp/train_batch2.j

# FIMUS Consistent-2 training for Consistent-1 Evaluation, more epochs


In [ ]:
!unzip -n -q {DRIVE}/FIMUSDataset/Consistent-2.zip -d {HOME}/dataset

In [ ]:
import os
import random

labels_path = '/content/dataset/Consistent-2/labels'
output_path = '/content/dataset/Consistent-2'

# Ratios for splitting the datasets
train_ratio = 1
val_ratio = 0

# test_ratio is implicitly determined

# Get all file names without their extensions
filenames = [os.path.splitext(file)[0] for file in os.listdir(labels_path) if os.path.isfile(os.path.join(labels_path, file))]

# Shuffle the list of filenames to ensure random distribution
random.shuffle(filenames)

# Calculate split indices
no_total_files = len(filenames)
train_end = int(no_total_files * train_ratio)
print(train_end)
print(no_total_files)

if(no_total_files == train_end):
  train_end-=1

val_end = train_end + int(no_total_files * val_ratio) +1

# Split the filenames
train_filenames = filenames[:train_end]
val_filenames = filenames[train_end:val_end]
test_filenames = filenames[val_end:]
print(val_filenames)

# Function to write filenames to a file
def write_filenames_to_file(filenames, file_path):
    with open(file_path, 'w') as file:
        for name in filenames:
            file.write(f'./images/{name}.jpg\n')

# Write the splits to their respective files
write_filenames_to_file(train_filenames, os.path.join(output_path, 'train.txt'))
write_filenames_to_file(val_filenames, os.path.join(output_path, 'val.txt'))
write_filenames_to_file(test_filenames, os.path.join(output_path, 'test.txt'))

print("Files have been split and saved successfully.")

In [ ]:
!cp /content/drive/MyDrive/FIMUSDataset/Consistent-2-20e.pt /content/weights/Consistent-2-20e.pt

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 30 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data-Consistent-2.yaml \
--cfg yolov9-e.yaml \
--project /content/Consistent-2-50e \
--single-cls \
--noval \
--weights /content/weights/Consistent-2-20e.pt \
--freeze 28

In [ ]:
results_saved_to = "Consistent-2-50e/exp"
!zip -r {DRIVE}/Consistent-2-50e.zip {HOME}/$results_saved_to

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 25 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data-Consistent-2.yaml \
--cfg yolov9-e.yaml \
--project /content/Consistent-2-75e \
--single-cls \
--noval \
--weights /content/Consistent-2-50e/exp/weights/best.pt \
--freeze 28

In [ ]:
results_saved_to = "Consistent-2-75e/exp"
!zip -r {DRIVE}/Consistent-2-75e.zip {HOME}/$results_saved_to

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 25 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data-Consistent-2.yaml \
--cfg yolov9-e.yaml \
--project /content/Consistent-2-100e \
--single-cls \
--noval \
--weights /content/Consistent-2-75e/exp/weights/best.pt \
--freeze 28

In [ ]:
results_saved_to = "Consistent-2-100e/exp"
!zip -r {DRIVE}/Consistent-2-100e.zip {HOME}/$results_saved_to

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 25 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data-Consistent-2.yaml \
--cfg yolov9-e.yaml \
--project /content/Consistent-2-125e \
--single-cls \
--noval \
--weights /content/Consistent-2-100e/exp/weights/best.pt \
--freeze 28

In [ ]:
results_saved_to = "Consistent-2-125e/exp"
!zip -r {DRIVE}/Consistent-2-125e.zip {HOME}/$results_saved_to

In [ ]:
%cd {HOME}
!python yolov9-masterthesis/train_dual.py \
--batch 10 \
--epochs 25 \
--img 640 \
--min-items 0 \
--data /content/yolov9-masterthesis/data-Consistent-2.yaml \
--cfg yolov9-e.yaml \
--project /content/Consistent-2-150e \
--single-cls \
--noval \
--weights /content/Consistent-2-125e/exp/weights/best.pt \
--freeze 28

In [ ]:
results_saved_to = "Consistent-2-150e/exp"
!zip -r {DRIVE}/Consistent-2-150e.zip {HOME}/$results_saved_to